In [1]:
# 크롤러를 만들기 전 필요한 도구들을 임포트합니다.
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup

csv_path = os.getenv("HOME") + "/aiffel/news_crawler/news_data_add.csv"
df = pd.read_table(csv_path, sep=',')

# 정규 표현식을 이용해서 한글 외의 문자는 전부 제거합니다.
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

# 중복된 샘플들을 제거합니다.
df.drop_duplicates(subset=['news'], inplace=True)

from konlpy.tag import Mecab  
tokenizer = Mecab()

#               precision    recall  f1-score   support

#        IT/과학       0.73      0.82      0.77       390
#           사회       0.80      0.81      0.80       439
#        생활/문화       0.76      0.66      0.71       455

#     accuracy                           0.76      1284
#    macro avg       0.76      0.76      0.76      1284
# weighted avg       0.76      0.76      0.76      1284

# from konlpy.tag import Komoran
# tokenizer = Komoran()

#               precision    recall  f1-score   support

#        IT/과학       0.72      0.83      0.77       390
#           사회       0.81      0.79      0.80       439
#        생활/문화       0.75      0.68      0.71       455

#     accuracy                           0.76      1284
#    macro avg       0.76      0.77      0.76      1284
# weighted avg       0.76      0.76      0.76      1284


# from konlpy.tag import Okt             
# tokenizer = Okt()

#               precision    recall  f1-score   support

#        IT/과학       0.72      0.83      0.77       390
#           사회       0.79      0.79      0.79       439
#        생활/문화       0.76      0.66      0.71       455

#     accuracy                           0.76      1284
#    macro avg       0.76      0.76      0.76      1284
# weighted avg       0.76      0.76      0.75      1284

# from konlpy.tag import Kkma
# tokenizer = Kkma()

#               precision    recall  f1-score   support

#        IT/과학       0.71      0.83      0.76       390
#           사회       0.82      0.80      0.81       439
#        생활/문화       0.75      0.66      0.70       455

#     accuracy                           0.76      1284
#    macro avg       0.76      0.76      0.76      1284
# weighted avg       0.76      0.76      0.76      1284


stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수',
             '무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스',
             '날', '무상', '원', '억', '도', '어', '나', '것', '살',
             '야', '만', '중', '라', '아', '기', '찬', '간', '상', '개', '곳', '어', '로', '합니다', '와',
             '년', '새', '서', '화', '공개', '환', '화', '서']

# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수입니다.
def preprocessing(data):
    text_data = []

    for index, sentence in enumerate(data):
        try:
            #- 토큰화
            temp_data = tokenizer.morphs(sentence)
            #- 불용어 제거
            temp_data = [word for word in temp_data if word not in stopwords]
            text_data.append(' '.join(temp_data))
        except Exception as e:
#             print(f"Error at index {index}: {sentence}\nError: {e}")
            text_data.append('')  # Append an empty string or some placeholder
            continue  # Continue with the next iteration

    return text_data

text_data = preprocessing(df['news'])

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

#- 훈련 데이터와 테스트 데이터를 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf = MultinomialNB().fit(X_train_tfidf, y_train)

def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

/tmp/ipykernel_228/54035449.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


              precision    recall  f1-score   support

       IT/과학       0.73      0.82      0.77       390
          사회       0.80      0.81      0.80       439
       생활/문화       0.76      0.66      0.71       455

    accuracy                           0.76      1284
   macro avg       0.76      0.76      0.76      1284
weighted avg       0.76      0.76      0.76      1284



# 회고

- stopwords 추가에 따른 성능 향상은 미미 하였습니다.
- Mecab외 Komoran, Okt, Kkma 사용하였으며, 소숫점 두자리 정도에서 성능차이가 발생 하였습니다.
- 초기에 2023년 11월 28일 데이터에 2024년 1월 10일 데이터를 추가하였으며 F1 5% 정도 성능이 향상 되었습니다.
- 가장 어려웠던 부분은 데이터 전처리 과정에서 오류가 발생한 부분을 확인하고 try - exception 처리하는 과정 이었습니다.
- pandas 학습을 통하여 데이터의 처리를 좀더 자유롭게 하면 좋을것 같습니다.